# Lakehouse Agent - Optional Cleanup

This notebook helps you clean up all AWS resources created by notebooks 00-06.

**⚠️ WARNING: This will delete all resources created during deployment!**

**What this notebook does:**
- Deletes Agent Runtime (from notebook 05)
- Deletes Gateway, Targets, OAuth Providers, and IAM Roles (from notebook 04)
- Deletes Interceptor Lambda (from notebook 04)
- Deletes MCP Server Runtime (from notebook 03)
- Deletes Cognito User Pool and users (from notebook 02)
- Deletes Athena database and tables (from notebook 01)
- Optionally deletes S3 bucket and data (from notebook 00)
- Deletes SSM parameters
- Deletes local configuration files

**Prerequisites:**
- AWS credentials configured
- Python 3.10 or later
- boto3 installed

In [ ]:
import boto3
import json
import time
from datetime import datetime

print("✅ Imports successful")

## Initialize AWS Session

In [ ]:
# Load AWS credentials and initialize session
from utils.notebook_init import init_aws

session, region, account_id = init_aws()

# Initialize AWS clients
ssm_client = session.client('ssm', region_name=region)
s3_client = session.client('s3', region_name=region)
athena_client = session.client('athena', region_name=region)
glue_client = session.client('glue', region_name=region)
cognito_client = session.client('cognito-idp', region_name=region)
lambda_client = session.client('lambda', region_name=region)
iam_client = session.client('iam', region_name=region)
bedrock_agent_client = session.client('bedrock-agentcore-control', region_name=region)

print(f'\n✅ AWS Session initialized')
print(f'   Account ID: {account_id}')
print(f'   Region: {region}')

## Step 1: Load Configuration from SSM

Load all configuration parameters to identify resources to delete.

In [ ]:
def get_ssm_parameter(name, default=None):
    """Get SSM parameter value, return default if not found"""
    try:
        response = ssm_client.get_parameter(Name=name)
        return response['Parameter']['Value']
    except ssm_client.exceptions.ParameterNotFound:
        return default
    except Exception as e:
        print(f"⚠️  Error reading {name}: {e}")
        return default

print("📋 Loading configuration from SSM Parameter Store...\n")

# Load all configuration
config = {
    'agent_runtime_arn': get_ssm_parameter('/app/lakehouse-agent/agent-runtime-arn'),
    'agent_runtime_id': get_ssm_parameter('/app/lakehouse-agent/agent-runtime-id'),
    'gateway_arn': get_ssm_parameter('/app/lakehouse-agent/gateway-arn'),
    'gateway_id': get_ssm_parameter('/app/lakehouse-agent/gateway-id'),
    'interceptor_lambda_arn': get_ssm_parameter('/app/lakehouse-agent/interceptor-lambda-arn'),
    'mcp_server_runtime_arn': get_ssm_parameter('/app/lakehouse-agent/mcp-server-runtime-arn'),
    'mcp_server_runtime_id': get_ssm_parameter('/app/lakehouse-agent/mcp-server-runtime-id'),
    'cognito_user_pool_id': get_ssm_parameter('/app/lakehouse-agent/cognito-user-pool-id'),
    'cognito_domain': get_ssm_parameter('/app/lakehouse-agent/cognito-domain'),
    'database_name': get_ssm_parameter('/app/lakehouse-agent/database-name'),
    's3_bucket_name': get_ssm_parameter('/app/lakehouse-agent/s3-bucket-name'),
}

# Display configuration
print("Resources found:")
for key, value in config.items():
    if value:
        display_value = value[:60] + '...' if len(value) > 60 else value
        print(f"  ✅ {key}: {display_value}")
    else:
        print(f"  ⏭️  {key}: Not found")

# Count resources
resource_count = sum(1 for v in config.values() if v)
print(f"\n📊 Found {resource_count} resources to clean up")

## Step 2: Delete Agent Runtime (from notebook 05)

Delete the Lakehouse Agent Runtime.

In [ ]:
print("🗑️  Deleting Agent Runtime...\n")

if config['agent_runtime_id']:
    try:
        bedrock_agent_client.delete_agent_runtime(
            agentRuntimeId=config['agent_runtime_id']
        )
        print(f"✅ Deleted Agent Runtime: {config['agent_runtime_id']}")
        print("   Waiting for deletion to complete...")
        time.sleep(10)
    except bedrock_agent_client.exceptions.ResourceNotFoundException:
        print(f"⏭️  Agent Runtime not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting Agent Runtime: {e}")
else:
    print("⏭️  No Agent Runtime found in configuration")

## Step 3: Delete Gateway (from notebook 04)

Delete the AgentCore Gateway.

In [ ]:
print("🗑️  Deleting Gateway...\n")

if config['gateway_id']:
    try:
        # First, delete all gateway targets
        print(f"Listing targets for gateway: {config['gateway_id']}")
        try:
            list_response = bedrock_agent_client.list_gateway_targets(
                gatewayIdentifier=config['gateway_id']
            )
            
            targets = list_response.get('items', [])
            if targets:
                print(f"  Found {len(targets)} target(s) to delete\n")
                for target in targets:
                    target_id = target['targetId']
                    target_name = target.get('name', 'unnamed')
                    try:
                        bedrock_agent_client.delete_gateway_target(
                            gatewayIdentifier=config['gateway_id'],
                            targetId=target_id
                        )
                        print(f"  ✅ Deleted target: {target_name} ({target_id})")
                    except Exception as e:
                        print(f"  ⚠️  Could not delete target {target_name}: {e}")
            else:
                print("  No targets found")
        except Exception as e:
            print(f"  ⚠️  Could not list targets: {e}")

        # Wait for targets to finish deleting
        if targets:
            print("\n⏳ Waiting for targets to finish deleting...")
            max_attempts = 12  # 12 attempts * 5 seconds = 60 seconds max
            for attempt in range(max_attempts):
                try:
                    list_response = bedrock_agent_client.list_gateway_targets(
                        gatewayIdentifier=config['gateway_id']
                    )
                    remaining_targets = list_response.get('items', [])

                    if not remaining_targets:
                        print("   ✅ All targets deleted successfully")
                        break

                    print(f"   Still {len(remaining_targets)} target(s) remaining... (attempt {attempt+1}/{max_attempts})")
                    time.sleep(5)
                except bedrock_agent_client.exceptions.ResourceNotFoundException:
                    print("   ✅ Gateway already deleted during target cleanup")
                    break
                except Exception as e:
                    print(f"   ⚠️  Error checking targets: {e}")
                    break
            else:
                print("   ⚠️  Timeout waiting for targets to delete, proceeding anyway...")

        # Now delete the gateway
        print(f"\nDeleting gateway: {config['gateway_id']}")
        bedrock_agent_client.delete_gateway(
            gatewayIdentifier=config['gateway_id']
        )
        print(f"✅ Deleted Gateway: {config['gateway_id']}")
        print("   Waiting for deletion to complete...")
        time.sleep(10)
    except bedrock_agent_client.exceptions.ResourceNotFoundException:
        print(f"⏭️  Gateway not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting Gateway: {e}")
else:
    print("⏭️  No Gateway found in configuration")

## Step 3.5: Delete OAuth2 Credential Providers

Delete OAuth2 credential providers created for Gateway-to-Runtime authentication.

**Important:** These providers store Cognito client credentials and must be deleted to prevent stale credentials from being reused.

In [ ]:
print("🗑️  Deleting OAuth2 Credential Providers...\n")

try:
    # List all OAuth2 credential providers
    response = bedrock_agent_client.list_oauth2_credential_providers()
    
    # Try different possible key names for the list
    providers = response.get('oauth2CredentialProviders', 
                           response.get('credentialProviders', 
                           response.get('items', [])))
    
    if providers:
        deleted_count = 0
        for provider in providers:
            provider_name = provider.get('name', 'unknown')
            
            # Only delete lakehouse-related providers
            if 'lakehouse' in provider_name.lower():
                try:
                    # Delete using the provider name
                    bedrock_agent_client.delete_oauth2_credential_provider(
                        name=provider_name
                    )
                    print(f"  ✅ Deleted OAuth provider: {provider_name}")
                    deleted_count += 1
                except Exception as e:
                    print(f"  ⚠️  Could not delete provider {provider_name}: {e}")
        
        if deleted_count > 0:
            print(f"\n✅ Deleted {deleted_count} OAuth2 provider(s)")
        else:
            print("\n⏭️  No lakehouse-related OAuth providers found")
    else:
        print("⏭️  No OAuth2 providers found")
        
except Exception as e:
    print(f"❌ Error listing OAuth2 providers: {e}")

## Step 3.6: Delete IAM Roles

Delete IAM roles created for the Gateway.

In [ ]:
print("🗑️  Deleting IAM Roles...\n")

# IAM role name for gateway
role_name = 'agentcore-lakehouse-gateway-role'

try:
    # Check if role exists
    iam_client.get_role(RoleName=role_name)
    
    print(f"Found IAM role: {role_name}")
    
    # Delete inline policies
    try:
        policy_names = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
        for policy_name in policy_names:
            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            print(f"  ✅ Deleted inline policy: {policy_name}")
    except Exception as e:
        print(f"  ⚠️  Error deleting inline policies: {e}")
    
    # Detach managed policies
    try:
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        for policy in attached_policies:
            iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
            print(f"  ✅ Detached managed policy: {policy['PolicyName']}")
    except Exception as e:
        print(f"  ⚠️  Error detaching managed policies: {e}")
    
    # Remove from instance profiles
    try:
        instance_profiles = iam_client.list_instance_profiles_for_role(RoleName=role_name)['InstanceProfiles']
        for profile in instance_profiles:
            iam_client.remove_role_from_instance_profile(
                InstanceProfileName=profile['InstanceProfileName'],
                RoleName=role_name
            )
            print(f"  ✅ Removed from instance profile: {profile['InstanceProfileName']}")
    except Exception as e:
        print(f"  ⚠️  Error removing from instance profiles: {e}")
    
    # Delete the role
    iam_client.delete_role(RoleName=role_name)
    print(f"\n✅ Deleted IAM role: {role_name}")
    
except iam_client.exceptions.NoSuchEntityException:
    print(f"⏭️  IAM role not found: {role_name}")
except Exception as e:
    print(f"❌ Error deleting IAM role: {e}")

## Step 4: Delete Interceptor Lambda (from notebook 04)

Delete the Gateway Interceptor Lambda function.

In [ ]:
print("🗑️  Deleting Interceptor Lambda...\n")

if config['interceptor_lambda_arn']:
    function_name = config['interceptor_lambda_arn'].split(':')[-1]
    try:
        lambda_client.delete_function(FunctionName=function_name)
        print(f"✅ Deleted Lambda function: {function_name}")
    except lambda_client.exceptions.ResourceNotFoundException:
        print(f"⏭️  Lambda function not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting Lambda function: {e}")
else:
    print("⏭️  No Interceptor Lambda found in configuration")

## Step 5: Delete MCP Server Runtime (from notebook 03)

Delete the MCP Server Runtime.

In [ ]:
print("🗑️  Deleting MCP Server Runtime...\n")

if config['mcp_server_runtime_id']:
    try:
        bedrock_agent_client.delete_agent_runtime(
            agentRuntimeId=config['mcp_server_runtime_id']
        )
        print(f"✅ Deleted MCP Server Runtime: {config['mcp_server_runtime_id']}")
        print("   Waiting for deletion to complete...")
        time.sleep(10)
    except bedrock_agent_client.exceptions.ResourceNotFoundException:
        print(f"⏭️  MCP Server Runtime not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting MCP Server Runtime: {e}")
else:
    print("⏭️  No MCP Server Runtime found in configuration")

## Step 6: Delete Local Configuration Files

Delete `.bedrock_agentcore.yaml` configuration files created during runtime deployment.

In [ ]:
import os
from pathlib import Path

print("🗑️  Deleting local configuration files...\n")

# Get notebook directory to build relative paths
notebook_dir = Path.cwd()

# Files to clean up
config_files = [
    notebook_dir / "deployment" / "mcp-lakehouse-server" / ".bedrock_agentcore.yaml",
    notebook_dir / "deployment" / "lakehouse-agent" / ".bedrock_agentcore.yaml"
]

deleted_count = 0
for file_path in config_files:
    if file_path.exists():
        try:
            file_path.unlink()
            print(f"✅ Deleted: {file_path.relative_to(notebook_dir)}")
            deleted_count += 1
        except Exception as e:
            print(f"❌ Error deleting {file_path.relative_to(notebook_dir)}: {e}")
    else:
        print(f"⏭️  Not found: {file_path.relative_to(notebook_dir)}")

if deleted_count > 0:
    print(f"\n✅ Deleted {deleted_count} configuration file(s)")
else:
    print("\n⏭️  No configuration files to delete")

## Step 6: Delete Cognito Resources (from notebook 02)

Delete Cognito User Pool, domain, and all users.

In [ ]:
print("🗑️  Deleting Cognito resources...\n")

if config['cognito_user_pool_id']:
    try:
        # First, try to get the user pool to check if domain exists
        try:
            pool_info = cognito_client.describe_user_pool(
                UserPoolId=config['cognito_user_pool_id']
            )
            pool_domain = pool_info['UserPool'].get('Domain')
            
            # If domain exists in the pool, delete it
            if pool_domain:
                print(f"Found domain in user pool: {pool_domain}")
                try:
                    cognito_client.delete_user_pool_domain(
                        Domain=pool_domain,
                        UserPoolId=config['cognito_user_pool_id']
                    )
                    print(f"✅ Deleted Cognito domain: {pool_domain}")
                    time.sleep(5)  # Wait for domain deletion to complete
                except cognito_client.exceptions.InvalidParameterException as e:
                    if "No such domain" in str(e):
                        print(f"⏭️  Domain already deleted")
                    else:
                        print(f"⚠️  Could not delete domain: {e}")
                except Exception as e:
                    print(f"⚠️  Could not delete domain: {e}")
            else:
                print("No domain configured for this user pool")
        except cognito_client.exceptions.ResourceNotFoundException:
            print(f"⏭️  User pool not found, skipping domain deletion")
        
        # Now delete the user pool
        cognito_client.delete_user_pool(
            UserPoolId=config['cognito_user_pool_id']
        )
        print(f"✅ Deleted Cognito User Pool: {config['cognito_user_pool_id']}")
    except cognito_client.exceptions.ResourceNotFoundException:
        print(f"⏭️  Cognito User Pool not found (may have been deleted already)")
    except cognito_client.exceptions.InvalidParameterException as e:
        if "domain configured" in str(e):
            print(f"❌ User pool still has a domain. Trying to find and delete it...")
            # Try with the domain from config
            if config.get('cognito_domain'):
                try:
                    cognito_client.delete_user_pool_domain(
                        Domain=config['cognito_domain'],
                        UserPoolId=config['cognito_user_pool_id']
                    )
                    print(f"✅ Deleted domain from config: {config['cognito_domain']}")
                    time.sleep(5)
                    # Try deleting user pool again
                    cognito_client.delete_user_pool(
                        UserPoolId=config['cognito_user_pool_id']
                    )
                    print(f"✅ Deleted Cognito User Pool: {config['cognito_user_pool_id']}")
                except Exception as e2:
                    print(f"❌ Still could not delete user pool: {e2}")
            else:
                print(f"❌ Error deleting Cognito User Pool: {e}")
        else:
            print(f"❌ Error deleting Cognito User Pool: {e}")
    except Exception as e:
        print(f"❌ Error deleting Cognito User Pool: {e}")
else:
    print("⏭️  No Cognito User Pool found in configuration")

## Step 7: Delete Athena Database and Tables (from notebook 01)

Delete Athena database and all tables.

In [ ]:
print("🗑️  Deleting Athena database and tables...\n")

if config['database_name']:
    try:
        # Get all tables in the database
        response = glue_client.get_tables(DatabaseName=config['database_name'])
        tables = response['TableList']
        
        # Delete each table
        for table in tables:
            table_name = table['Name']
            try:
                glue_client.delete_table(
                    DatabaseName=config['database_name'],
                    Name=table_name
                )
                print(f"  ✅ Deleted table: {table_name}")
            except Exception as e:
                print(f"  ⚠️  Could not delete table {table_name}: {e}")
        
        # Delete the database
        glue_client.delete_database(Name=config['database_name'])
        print(f"\n✅ Deleted database: {config['database_name']}")
    except glue_client.exceptions.EntityNotFoundException:
        print(f"⏭️  Database not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting database: {e}")
else:
    print("⏭️  No Athena database found in configuration")

## Step 8: Delete S3 Bucket (from notebook 00)

**⚠️ WARNING: This will permanently delete all data in the S3 bucket!**

Set `DELETE_S3_BUCKET = True` to enable S3 bucket deletion.

In [ ]:
# Set to True to delete the S3 bucket and all its contents
DELETE_S3_BUCKET = False  # Change to True to enable deletion

print("🗑️  S3 Bucket cleanup...\n")

if not DELETE_S3_BUCKET:
    print("⏭️  S3 bucket deletion is DISABLED")
    print("   Set DELETE_S3_BUCKET = True to enable")
    print(f"   Bucket: {config['s3_bucket_name']}")
elif config['s3_bucket_name']:
    try:
        bucket_name = config['s3_bucket_name']
        
        # List and delete all objects
        print(f"Deleting all objects in bucket: {bucket_name}")
        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket_name)
        
        delete_count = 0
        for page in pages:
            if 'Contents' in page:
                objects = [{'Key': obj['Key']} for obj in page['Contents']]
                s3_client.delete_objects(
                    Bucket=bucket_name,
                    Delete={'Objects': objects}
                )
                delete_count += len(objects)
        
        print(f"  ✅ Deleted {delete_count} objects")
        
        # Delete the bucket
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"\n✅ Deleted S3 bucket: {bucket_name}")
    except s3_client.exceptions.NoSuchBucket:
        print(f"⏭️  S3 bucket not found (may have been deleted already)")
    except Exception as e:
        print(f"❌ Error deleting S3 bucket: {e}")
else:
    print("⏭️  No S3 bucket found in configuration")

## Step 9: Delete SSM Parameters

Delete all SSM parameters created during deployment.

In [ ]:
print("🗑️  Deleting SSM parameters...\n")

# Get all parameters with the lakehouse-agent prefix
try:
    paginator = ssm_client.get_paginator('describe_parameters')
    pages = paginator.paginate(
        ParameterFilters=[
            {
                'Key': 'Name',
                'Option': 'BeginsWith',
                'Values': ['/app/lakehouse-agent/']
            }
        ]
    )
    
    parameters_to_delete = []
    for page in pages:
        for param in page['Parameters']:
            parameters_to_delete.append(param['Name'])
    
    if parameters_to_delete:
        print(f"Found {len(parameters_to_delete)} parameters to delete:\n")
        
        # Delete parameters in batches of 10 (AWS limit)
        for i in range(0, len(parameters_to_delete), 10):
            batch = parameters_to_delete[i:i+10]
            try:
                ssm_client.delete_parameters(Names=batch)
                for param in batch:
                    print(f"  ✅ Deleted: {param}")
            except Exception as e:
                print(f"  ❌ Error deleting batch: {e}")
        
        print(f"\n✅ Deleted {len(parameters_to_delete)} SSM parameters")
    else:
        print("⏭️  No SSM parameters found with /app/lakehouse-agent/ prefix")
        
except Exception as e:
    print(f"❌ Error deleting SSM parameters: {e}")

## Summary

Review the cleanup results above.

In [ ]:
print("\n" + "="*70)
print("🎉 CLEANUP COMPLETE")
print("="*70)

print("\n✅ Resources cleaned up:")
print("   • Agent Runtime (notebook 05)")
print("   • Gateway & Targets (notebook 04)")
print("   • OAuth2 Credential Providers")
print("   • IAM Roles (Gateway)")
print("   • Interceptor Lambda (notebook 04)")
print("   • MCP Server Runtime (notebook 03)")
print("   • Local Configuration Files (.bedrock_agentcore.yaml)")
print("   • Cognito User Pool (notebook 02)")
print("   • Athena Database & Tables (notebook 01)")
print("   • SSM Parameters (notebook 00)")

if DELETE_S3_BUCKET:
    print("   • S3 Bucket & Data (notebook 00)")
else:
    print("   ⏭️  S3 Bucket (skipped - set DELETE_S3_BUCKET=True to delete)")

print("\n📋 Manual cleanup (if needed):")
print("   • CloudWatch Log Groups: /aws/bedrock-agentcore/runtime/*")
print("   • CloudWatch Log Groups: /aws/lambda/lakehouse-*")
print("   • ECR Repositories (if created)")

print("\n" + "="*70)